# Benchmarking different quantum compilers


## Load benchmarks

In [92]:
import os
from pathlib import Path


def trim_benchmark_files(benchmark_files):
    trimmed_benchmarks_files = []
    for benchmark_file in benchmark_files:
        if not (str(benchmark_file) == "temp_in.qasm" or str(benchmark_file) == "tmp.qasm" or "pyvoqc" in str(
                benchmark_file)):  # Ignore compiler artifacts
            trimmed_benchmarks_files.append(benchmark_file)
    return trimmed_benchmarks_files


benchmark_files = trim_benchmark_files(list(Path(".").rglob("*.qasm")))

## Prepare compilers

In [93]:
compilers = dict()

## QisKit

In [94]:
from qiskit.compiler import transpile

In [95]:
def optimize_qiskit(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    opt_qc = transpile(qc, optimization_level=3, approximation_degree=1.0,
                       basis_gates=['rz', 'rx', 'h', 't', 'tdag', 's', 'cx', 'id']) # For some reason qiskit cannot unroll id to Rz(0)
    return opt_qc.qasm()


compilers['qiskit'] = optimize_qiskit

## PyZX

In [96]:
import pyzx as zx

In [97]:
def optimize_pyzx(qasm):
    c = zx.Circuit.from_qasm(qasm)
    c = c.to_graph()
    zx.simplify.full_reduce(c)
    c.normalize()
    c_opt = zx.extract_circuit(c.copy(), quiet=True).to_basic_gates()
    return c_opt.to_qasm()


compilers['pyzx'] = optimize_pyzx

## VOQC

To install voqc run
```shell
git submodule update --remote
cd pyvoqc
opam pin voqc https://github.com/inQWIRE/mlvoqc.git#mapping
./install.sh
```
Then copy pyvoqc/lib to your site packages pyvoqc folder

In [98]:
from qiskit import QuantumCircuit
from pyvoqc.qiskit.voqc_pass import voqc_pass_manager

In [99]:
def optimize_voqc(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    vpm = voqc_pass_manager(post_opts=["optimize"])
    qc_opt = vpm.run(qc)
    return qc_opt.qasm()


compilers['voqc'] = optimize_voqc

## $\mid$tket$\rangle$

In [100]:
from pytket.qasm import circuit_from_qasm_str, circuit_to_qasm_str
from pytket.passes import SequencePass, RemoveRedundancies, FullPeepholeOptimise, auto_rebase_pass
from pytket.predicates import CompilationUnit
from pytket import OpType

In [101]:
def optimize_tket(qasm):
    qc = circuit_from_qasm_str(qasm)
    gates = {OpType.Rz, OpType.Rx, OpType.H, OpType.T, OpType.Tdg, OpType.S, OpType.CX}
    rebase = auto_rebase_pass(gates)
    seqpass = SequencePass(
        [
            FullPeepholeOptimise(),
            RemoveRedundancies(),
            rebase,
        ])
    cu = CompilationUnit(qc)
    seqpass.apply(cu)
    qc_opt = cu.circuit
    return circuit_to_qasm_str(qc_opt)


compilers['tket'] = optimize_tket

## Staq

In [102]:
import pystaq


In [103]:
def optimize_staq(qasm):
    qc = pystaq.parse_str(qasm)
    pystaq.simplify(qc)
    pystaq.rotation_fold(qc)
    pystaq.simplify(qc)
    pystaq.cnot_resynth(qc)
    pystaq.simplify(qc)
    return str(qc)


compilers['staq'] = optimize_staq


# Run Benchmarks

## Gate counting

We count gates by finding them in the circuit representation.
Though, we assume a error corrected gate set, meaning we will synthesize any rotations that are not of form $c \times \frac{pi}{4}$ using Peter Sellinger's gridsynth (https://www.mathstat.dal.ca/~selinger/newsynth/)

If you don't run macOS you might need to replace the binary

In [104]:
import subprocess


def gridsynth(theta):
    return subprocess.getoutput(f"./gridsynth {theta} -e 10e-8")

In [105]:
from math import pi


def count_gates(qasm, gate_names):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in gate_names:
            count += 1
    return count


def approx(a, b):
    return abs(a - b) < 10e-4


def count_gates_synth(theta):
    synth = gridsynth(theta)
    return len(synth) - synth.count('W')


def count_t_synth(theta):
    synth = gridsynth(theta)
    return synth.count('T')


def count_rot_z_t(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in ['rz', 'rx', 'u1']:
            theta = gate.params[0]
            if abs(theta % (pi / 4)) < 10e-5:
                count += approx(pi / 4, abs(theta % (pi / 2)))
            else:
                count += count_t_synth(theta)

    return count


def count_t_gates(qasm):
    return count_gates(qasm, ['t', 'tdag']) + count_rot_z_t(qasm)


def count_cnot_gates(qasm):
    return count_gates(qasm, ['cx', 'cz'])


def count_total_gates(qasm):
    qc = QuantumCircuit.from_qasm_str(qasm)
    gates = list(qc)
    count = 0
    for gate in gates:
        gate = gate[0]
        if gate.name in ['rz', 'rx', 'u1']:
            theta = gate.params[0]
            if abs(theta % (pi / 4)) < 10e-5:
                count += 1
            else:
                count += count_gates_synth(theta)
        else:
            count += 1
    return count


In [106]:
from qiskit.circuit.library import MCMT

# Test
testing = False
if testing:
    qc = QuantumCircuit(2)
    qc.rz(0.132312, 0)
    qc.t(0)
    qc.h(1)
    qc.cnot(0, 1)
    qc.t(0)
    qc.cnot(0, 1)
    qc.h(0)
    qc.t(0)
    qc.t(1)
    qc.s(1)
    qc.cnot(1, 0)
    qc.rx(pi / 4, 1)

    print("Original")
    print(qc.draw())

    qasm = qc.qasm()

    for compiler_name in compilers:
        print(compiler_name)
        qasm_opt = compilers[compiler_name](qasm)
        qc_opt = QuantumCircuit.from_qasm_str(qasm_opt)
        print(qc_opt.draw())
        print(count_t_gates(qasm_opt))


In [107]:
import time
import pickle

In [108]:
# Create place to store files concerning optimiation and results
results_path = "results"
if not os.path.exists(results_path):
    os.makedirs(results_path)

In [109]:
def generate_result(qasm, time_taken=0):
    result = dict()
    result['qasm'] = qasm
    result['time'] = time_taken
    if not qasm == "":
        result['cnot_count'] = count_cnot_gates(qasm)
        result['t_count'] = count_t_gates(qasm)
        result['total_count'] = count_total_gates(qasm)
    return result

### Benchmark each compiler on its own and cache results
Will also generate baseline for non-optimized benchmarks

In [110]:
import signal


class timeout:
    def __init__(self, seconds=1, error_message='Timeout'):
        self.seconds = seconds
        self.error_message = error_message

    def handle_timeout(self, _, __):
        raise TimeoutError(self.error_message)

    def __enter__(self):
        signal.signal(signal.SIGALRM, self.handle_timeout)
        signal.alarm(self.seconds)

    def __exit__(self, _, __, ___):
        signal.alarm(0)

In [111]:
from qiskit.qasm import QasmError

In [112]:
enable_large_benchmarks = False

In [23]:
for benchmark_file in benchmark_files:
    path_no_slash = str(benchmark_file).replace("/", "-")
    print(benchmark_file)
    if (not enable_large_benchmarks) and os.path.getsize(benchmark_file) > 100 * 1000:  # 100 KB:
        print("Benchmark too large, skipping")
        continue
    try:
        benchmark = QuantumCircuit.from_qasm_file(benchmark_file)
            .decompose().qasm()  # Remove custom gate declarations since tket and voqc don't play nicely with them
    except QasmError as e:
        print(e)
        continue
    assert not benchmark == ""
    base_pkl_path = f"{results_path}/result_base_{path_no_slash}.pkl"
    if not os.path.exists(base_pkl_path):
        print("Establishing baseline")
        baseline = generate_result(benchmark)
        pickle.dump(baseline, open(base_pkl_path, 'wb'))
        print("Established baseline")
    else:
        print("Baseline already exists, continuing...")
    for compiler_name in compilers:
        print(f"Using {compiler_name} to optimize {benchmark_file}")
        pkl_path = f"{results_path}/result_{compiler_name}_{path_no_slash}.pkl"
        if os.path.exists(pkl_path):
            if compiler_name != "pyzx" or os.path.getmtime(pkl_path) >= 1652205600:
                print("Found cached results file, skipping computation")
                continue
        try:
            with timeout(60 * 15):
                start = time.perf_counter()
                opt_qasm = compilers[compiler_name](benchmark)
                stop = time.perf_counter()
                time_taken = stop - start
                print(f"Completed in {time_taken} sec")
        except TimeoutError:
            print("Timed out after 15min")
            time_taken = -1
            opt_qasm = ""
        result = generate_result(opt_qasm, time_taken)
        pickle.dump(result, open(pkl_path, 'wb'))


Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
Baseline already exists, continuing...
Using qiskit to optimize Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
Found cached results file, skipping computation
Using pyzx to optimize Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
Found cached results file, skipping computation
Using voqc to optimize Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
Found cached results file, skipping computation
Using tket to optimize Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
Found cached results file, skipping computation
Using staq to optimize Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
Found cached results file, skipping computation
Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof_5.qasm
Baseline already exists, continuing...
Using qiskit to optimize Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof_5.qasm
Found cached results file, skipping computation
Using pyzx to optimize Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof

### Use cached results to benchmark compiler juxtaposition

In [113]:
# Benchmark individually
for compiler_name_1 in compilers:
    for compiler_name_2 in compilers:
        key = compiler_name_1 + '%' + compiler_name_2
        if compiler_name_1 == compiler_name_2:
            continue
        for benchmark_file in benchmark_files:
            path_no_slash = str(benchmark_file).replace("/", "-")
            print(benchmark_file)
            if (not enable_large_benchmarks) and os.path.getsize(benchmark_file) > 100 * 1000:  # 100 KB:
                print("Benchmark too large, skipping")
                continue
            print(f"Using {compiler_name_2} to optimize {benchmark_file} previously optimized by {compiler_name_1}")
            pkl_path = f"{results_path}/result_{compiler_name_1}_{compiler_name_2}_{path_no_slash}.pkl"
            if os.path.exists(pkl_path):
                if  not (compiler_name_1 == "voqc" and compiler_name_2 == "qiskit"):
                    print("Found cached results file, skipping computation")
                    continue
            prior_result_path = f"{results_path}/result_{compiler_name_1}_{path_no_slash}.pkl"
            if not os.path.exists(prior_result_path):
                print("Missing prior file, skipping")
                continue
            prior_result = pickle.load(open(prior_result_path, 'rb'))
            if prior_result['time'] == -1:
                print("Prior benchmark did not complete successfully, skipping")
                continue
            opt_benchmark = prior_result['qasm']
            assert not opt_benchmark == ""
            print(f"{compiler_name_1} + {compiler_name_2}")
            try:
                with timeout(60 * 15):
                    start = time.perf_counter()
                    opt_qasm = compilers[compiler_name_2](opt_benchmark)
                    stop = time.perf_counter()
                    time_taken = stop - start
                    print(f"Completed in {time_taken} sec")
            except Exception as e:
                print(e)
                opt_qasm = ''
                time_taken = -1
            result = generate_result(opt_qasm, time_taken)
            pickle.dump(result, open(pkl_path, 'wb'))

Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm
Using pyzx to optimize Nam-benchmarks/Arithmetic_and_Toffoli/adder_8.qasm previously optimized by qiskit
Found cached results file, skipping computation
Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof_5.qasm
Using pyzx to optimize Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof_5.qasm previously optimized by qiskit
Found cached results file, skipping computation
Nam-benchmarks/Arithmetic_and_Toffoli/gf2^4_mult.qasm
Using pyzx to optimize Nam-benchmarks/Arithmetic_and_Toffoli/gf2^4_mult.qasm previously optimized by qiskit
Found cached results file, skipping computation
Nam-benchmarks/Arithmetic_and_Toffoli/gf2^128_mult.qasm
Benchmark too large, skipping
Nam-benchmarks/Arithmetic_and_Toffoli/mod_mult_55.qasm
Using pyzx to optimize Nam-benchmarks/Arithmetic_and_Toffoli/mod_mult_55.qasm previously optimized by qiskit
Found cached results file, skipping computation
Nam-benchmarks/Arithmetic_and_Toffoli/barenco_tof_4.qasm
Using pyzx to o

## Process results

In [114]:
def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [125]:
def generate_t_table_entry(pkl_path):
    if not os.path.exists(pkl_path):
        return 'X & '
    result = pickle.load(open(pkl_path, 'rb'))
    if result['time'] == -1:
        return 'X & '
    return str(result['t_count']) + ' & '


def process_benchmark_file_name(benchmark_file):
    name = ''
    if str(benchmark_file).lower().startswith("nam"):
        name += 'Nam'
    else:
        name += 'staq'
    name += '-' + str(benchmark_file).split('/')[-1]
    return name.replace('.qasm', '')


def process_benchmark_t_gates(benchmark_file, compiler_name_1=''):
    path_no_slash = str(benchmark_file).replace("/", "-")
    pkl_path_base = f"{results_path}/result_base_{path_no_slash}.pkl"
    row = process_benchmark_file_name(benchmark_file).replace('_','\\_').replace('^','\\^{}') + " & "
    if compiler_name_1 == '':
        row += generate_t_table_entry(pkl_path_base)
        for compiler_name in compilers:
            pkl_path = f"{results_path}/result_{compiler_name}_{path_no_slash}.pkl"
            row += generate_t_table_entry(pkl_path)
    else:
        for compiler_name_2 in compilers:
            if compiler_name_1 == compiler_name_2:
                continue
            pkl_path = f"{results_path}/result_{compiler_name_1}_{compiler_name_2}_{path_no_slash}.pkl"
            row += generate_t_table_entry(pkl_path)
    row = row[:-2] + ' \\\\'
    return row


def gen_header_row(compiler_name_1=''):
    if compiler_name_1 == '':
        header_row = "Benchmark & \\textbf{Base} & "
        for compiler_name in compilers:
            header_row += f"\\{compiler_name.lower()} & "
    else:
        header_row = "Benchmark & "
        for compiler_name_2 in compilers:
            if compiler_name_1 == compiler_name_2:
                continue
            header_row += f"\\{compiler_name_1.lower()}-\\{compiler_name_2.lower()} & "
    header_row = header_row[:-2] + '\\\\'
    return header_row


def gen_table_definition(compiler_name_1=''):
    table_def = "\\begin{tabular}{| l |"
    if compiler_name_1 == '':
        table_def += 'c |'
        for _ in compilers:
            table_def += " c |"
    else:
        for compiler_name_2 in compilers:
            if compiler_name_1 == compiler_name_2:
                continue
            table_def += " c |"
    table_def += "}"
    return table_def


def add_line(str, add):
    return str + '\n' + add


def add_line_indent(str, add):
    return str + '\n\t' + add


def gen_table():
    table = "\\begin{center}"
    compilers_names = [''] + list(compilers.keys())
    for compiler_name_1 in compilers_names:
        for chunk in chunk_list(benchmark_files, 70):
            table = add_line(table, "\\thispagestyle{empty}")
            table = add_line(table, gen_table_definition(compiler_name_1))
            table = add_line_indent(table, "\\hline")
            table = add_line_indent(table, gen_header_row(compiler_name_1))
            table = add_line_indent(table, "\\hline")
            for benchmark_file in chunk:
                path_no_slash = str(benchmark_file).replace("/", "-")
                pkl_path_base = f"{results_path}/result_base_{path_no_slash}.pkl"
                if not os.path.exists(pkl_path_base):
                    continue
                entry = process_benchmark_t_gates(benchmark_file, compiler_name_1)
                table = add_line_indent(table, entry)
            table = add_line_indent(table, "\\hline")
            table = add_line(table, "\end{tabular}\\\\")
            table = add_line(table, "\\newpage")
    table = add_line(table, "\\end{center}")
    return table



In [126]:
print(gen_table())

\begin{center}
\thispagestyle{empty}
\begin{tabular}{| l |c | c | c | c | c | c |}
	\hline
	Benchmark & \textbf{Base} & \qiskit & \pyzx & \voqc & \tket & \staq \\
	\hline
	Nam-adder\_8 & 513 & 473 & 173 & 217 & 17420 & 247  \\
	Nam-barenco\_tof\_5 & 108 & 106 & 40 & 44 & 3591 & 48  \\
	Nam-gf2\^{}4\_mult & 144 & 128 & 68 & 68 & 5431 & 242  \\
	Nam-mod\_mult\_55 & 63 & 63 & 35 & 39 & 1451 & 40  \\
	Nam-barenco\_tof\_4 & 72 & 70 & 28 & 30 & 2152 & 33  \\
	Nam-gf2\^{}32\_mult & 9216 & 8192 & 4128 & 4128 & 210434 & 5171  \\
	Nam-csum\_mux\_9 & 252 & 236 & 84 & 140 & 6524 & 140  \\
	Nam-gf2\^{}9\_mult & 729 & 657 & 351 & 351 & 21074 & 611  \\
	Nam-rc\_adder\_6 & 99 & 97 & 47 & 71 & 2546 & 64  \\
	Nam-barenco\_tof\_3 & 36 & 34 & 16 & 16 & 1426 & 18  \\
	Nam-barenco\_tof\_10 & 288 & 286 & 100 & 114 & 11221 & 123  \\
	Nam-mod\_adder\_1024 & 2565 & 2673 & 1011 & 1003 & 99153 & 1379  \\
	Nam-qcla\_com\_7 & 261 & 247 & 95 & 95 & 7656 & 190  \\
	Nam-mod5\_4 & 36 & 30 & 8 & 16 & 1254 & 17  \\
	Nam-

In [117]:
def no_of_tgates(pkl_path, base):
    if not os.path.exists(pkl_path):
        return base
    result = pickle.load(open(pkl_path, 'rb'))
    if result['time'] == -1:
        return base
    return result['t_count']


def count_total_t_gates():
    total_t = dict()
    base_key = "\\textbf{Base}"
    total_t[base_key] = 0
    for compiler_name_1 in compilers:
        total_t[f"\\{compiler_name_1}"] = 0
        for compiler_name_2 in compilers:
            if compiler_name_1 == compiler_name_2:
                continue
            total_t[f"\\{compiler_name_1}-\\{compiler_name_2}"] = 0
    for benchmark_file in benchmark_files:
        if str(benchmark_file) == "temp_in.qasm" or str(benchmark_file) == "tmp.qasm":  # Ignore compiler artifacts
            continue
        if "pyvoqc" in str(benchmark_file):  # Ignore unit tests
            continue
        path_no_slash = str(benchmark_file).replace("/", "-")
        pkl_path_base = f"{results_path}/result_base_{path_no_slash}.pkl"
        if not os.path.exists(pkl_path_base):
            continue
        base_result = pickle.load(open(pkl_path_base, 'rb'))
        base_count = base_result['t_count']
        total_t[base_key] += base_count
        for compiler_name in compilers:
            pkl_path = f"{results_path}/result_{compiler_name}_{path_no_slash}.pkl"
            total_t[f"\\{compiler_name}"] += no_of_tgates(pkl_path, base_count)
        for compiler_name_1 in compilers:
            for compiler_name_2 in compilers:
                if compiler_name_1 == compiler_name_2:
                    continue
                pkl_path_1 = f"{results_path}/result_{compiler_name_1}_{path_no_slash}.pkl"
                compiler1_t_count = no_of_tgates(pkl_path_1, base_count)
                pkl_path = f"{results_path}/result_{compiler_name_1}_{compiler_name_2}_{path_no_slash}.pkl"
                total_t[f"\\{compiler_name_1}-\\{compiler_name_2}"] += no_of_tgates(pkl_path, compiler1_t_count)
    return total_t


def gen_tables_total_t_gates():
    total_t = count_total_t_gates()
    tables = "\\begin{center}"
    chunks = chunk_list(list(total_t.keys())[1:], 5)
    first = True
    for chunk in chunks:
        if first:
            chunk.insert(0, list(total_t.keys())[0])  # Put base into first table
        first = False
        tables = add_line(tables, "\\begin{tabular}{|")
        for i in range(len(chunk)):
            tables += " c |"
        tables += "}\n\t\\hline\n\t"
        for key in chunk:
            tables += key + " &"
        tables = tables[:-2] + '\\\\\\hline\n\t'
        for key in chunk:
            tables += str(total_t[key]) + " &"
        tables = tables[:-2] + '\\\\\\hline'
        tables = add_line(tables, "\\end{tabular}\\\\\\vspace{12pt}\n")
    return tables + "\n\\end{center}"



In [118]:
print(gen_tables_total_t_gates())

\begin{center}
\begin{tabular}{| c | c | c | c | c | c |}
	\hline
	\textbf{Base} &\qiskit &\qiskit-\pyzx &\qiskit-\voqc &\qiskit-\tket &\qiskit-\staq\\\hline
	236765 &167760 &164998 &51738 &2421171 &161171\\\hline
\end{tabular}\\\vspace{12pt}

\begin{tabular}{| c | c | c | c | c |}
	\hline
	\pyzx &\pyzx-\qiskit &\pyzx-\voqc &\pyzx-\tket &\pyzx-\staq\\\hline
	161573 &101780 &154038 &603472 &615504\\\hline
\end{tabular}\\\vspace{12pt}

\begin{tabular}{| c | c | c | c | c |}
	\hline
	\voqc &\voqc-\qiskit &\voqc-\pyzx &\voqc-\tket &\voqc-\staq\\\hline
	41311 &92278 &157092 &1356671 &40346\\\hline
\end{tabular}\\\vspace{12pt}

\begin{tabular}{| c | c | c | c | c |}
	\hline
	\tket &\tket-\qiskit &\tket-\pyzx &\tket-\voqc &\tket-\staq\\\hline
	2163229 &2216292 &824942 &1001669 &1957346\\\hline
\end{tabular}\\\vspace{12pt}

\begin{tabular}{| c | c | c | c | c |}
	\hline
	\staq &\staq-\qiskit &\staq-\pyzx &\staq-\voqc &\staq-\tket\\\hline
	127311 &159467 &140136 &41918 &1267205\\\hline
\end{tab